In [1]:
import pandas as pd
from sklearn.utils import shuffle
from pandas import DataFrame
import numpy as np
from IPython.core.debugger import set_trace

In [ ]:
! python -m pip install pymongo==3.7.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 628 kB 13.8 MB/s 
  Created wheel for pymongo: filename=pymongo-3.7.2-cp38-cp38-linux_x86_64.whl size=415791 sha256=d12917e288722cafc34ae0b2dd792fd921936b4b2ef169fc02bd19c39f05115f
  Stored in directory: /root/.cache/pip/wheels/28/62/b5/ede9674d1415d2c15c3e805e6cc7debfcdf380105da0887776
Successfully built pymongo


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
from joblib import load
gpc_loaded = load('/content/drive/MyDrive/Colab Notebooks/modelos/localizacao.model')
gpc_loaded

In [ ]:
import datetime                           
import pymongo
from pymongo import MongoClient
# uri (uniform resource identifier) define os parêmtros de conexão com o banco de dados
uri = '' #IMPORTANTE: colocar aqui a url do banco de dados mongodb
# conecta à coleção alvo do mongoose
cluster = MongoClient( uri )
db = cluster["inteli-iot"]
collection = db["locations"]
training_collection = db["locationsTrainingData"]


In [ ]:
df = pd.DataFrame(list(collection.find({'room': {'$exists': False}})))
locations_copy = df.copy()

In [ ]:
formatedTotalSignalsDf = pd.DataFrame()
for index, row in df.iterrows():
  signalsDf = pd.DataFrame(row.signals)
  signalsDf = signalsDf.drop('_id', axis=1)
  macAddressArray = signalsDf['macAddress']
  strengthArray = signalsDf['strength']

  formatedDfData = {}
  for signalsIndex, signalsRow in signalsDf.iterrows():
    formatedDfData[signalsRow.macAddress] = signalsRow.strength 

  formatedSignalsDf = pd.DataFrame(formatedDfData, [index])
  
  formatedTotalSignalsDf = pd.concat([formatedTotalSignalsDf, formatedSignalsDf])
    
formatedTotalSignalsDf
df = pd.concat([df, formatedTotalSignalsDf], axis=1)


In [ ]:
#Exclusão das colunas que não serão utilizadas
df = df.drop('signals', axis=1)
df = df.drop('createdAt', axis = 1)
id_column = df['_id']
df = df.drop('_id', axis=1)
df = df.drop('deviceId', axis=1)
df = df.drop('__v', axis = 1)
df = df.drop('updatedAt', axis = 1)
if 'battery' in df.columns:
  df = df.drop('battery', axis = 1)
  

In [ ]:
#Preencher NaN com 0
df = df.fillna(0)
df

In [ ]:
training_df = pd.DataFrame(list(training_collection.find({'room': {'$exists': True}})))
training_df

In [ ]:
formatedTotalSignalsDf = pd.DataFrame()
for index, row in training_df.iterrows():
  signalsDf = pd.DataFrame(row.signals)
  signalsDf = signalsDf.drop('_id', axis=1)
  macAddressArray = signalsDf['macAddress']
  strengthArray = signalsDf['strength']

  formatedDfData = {}
  for signalsIndex, signalsRow in signalsDf.iterrows():
    formatedDfData[signalsRow.macAddress] = signalsRow.strength 

  formatedSignalsDf = pd.DataFrame(formatedDfData, [index])
  
  formatedTotalSignalsDf = pd.concat([formatedTotalSignalsDf, formatedSignalsDf])
    
training_df = pd.concat([training_df, formatedTotalSignalsDf], axis=1)


In [ ]:
#Exclusão das colunas que não serão utilizadas
training_df = training_df.drop('signals', axis=1)
training_df = training_df.drop('createdAt', axis = 1)
id_column = training_df['_id']
training_df = training_df.drop('_id', axis=1)
training_df = training_df.drop('deviceId', axis=1)
training_df = training_df.drop('__v', axis = 1)
training_df = training_df.drop('updatedAt', axis = 1)
if 'battery' in training_df.columns:
  training_df = training_df.drop('battery', axis = 1)
training_df = training_df.drop('room', axis = 1)

In [ ]:
training_df.shape

In [ ]:
df.shape

In [ ]:
missing_columns = training_df.columns.difference(df.columns)
missing_columns

In [ ]:
missing_columns_2 = df.columns.difference(training_df.columns)
len(missing_columns_2) 

In [ ]:
# df[missing_columns] = df[missing_columns].fillna(0)
for column in missing_columns:
  df[column] = 0

for column in missing_columns_2:
  df.drop(column, axis=1, inplace=True)

In [ ]:
training_df.shape

In [ ]:
y_pred = gpc_loaded.predict(df)
y_pred

In [ ]:
locations_copy['room'] = y_pred
locations_copy

In [ ]:
# for index, row in locations_copy.iterrows():
#   myquery = { "_id": row._id }
#   newvalues = { "$set": { "room": row.room } }
#   collection.update_one(myquery, newvalues)